In [1]:
#Import libraries
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Range of years
years = np.arange(2006,2019).tolist()

In [3]:
#Check to see if the website allows scraping
url = 'https://fred.stlouisfed.org/release/tables?rid=330&eid=391444'
page = requests.get(url)
page

<Response [200]>

In [15]:
year_str = '2006'
full_link = url+'&od='+year_str+'-01-01#'
page = requests.get(full_link)


#Grab the components - make the soup
soup = BeautifulSoup(page.text,'lxml')


# #Grab the states and make a dataframe
states = soup.find_all('span',class_='fred-rls-elm-nm')



states_list = []
for i in states:
    state_name = i.text
    states_list.append(state_name)

states_df = pd.DataFrame(states_list,columns=['State'])
states_df['index'] = np.arange(len(states_df))
states_df
# #states_df['Year'] = str(i)

#Grab the population values and make a dataframe
bachs = soup.find_all('td',class_='fred-rls-elm-vl-td')


bach_list = []
for i in bachs:
    bach_num = i.text
    bach_list.append(bach_num)



bach_df = pd.DataFrame(bach_list,columns=['Bach_Perc'])


bach_df['Bach_Perc'] = bach_df['Bach_Perc'].replace('\n',"",regex=True).str.strip()
bach_df = bach_df.iloc[::3, :]
bach_df['index'] = np.arange(len(bach_df))
# #pop_df['Year'] = str(i)


df = pd.merge(
        states_df,
        bach_df,
        how='left',
        on=['index']
)
    
df['Year'] = year_str
df.tail()

# #     for i in list_of_dfs
# #     full_df = pd.concat(list_of_dfs, ignore_index = True)

,State,index,Bach_Perc,Year
46,Virginia,46,32.7,2006
47,Washington,47,30.5,2006
48,West Virginia,48,16.5,2006
49,Wisconsin,49,25.1,2006
50,Wyoming,50,22.7,2006


In [16]:
#Set up dataframe
df = pd.DataFrame(
    {
        'State':[''], 
        'Year':[''],
        'Bach_Perc':['']
    }
)

list_of_dfs=[]

#For each year - run through this process
for i in years:
    full_link = url+'&od='+str(i)+'-01-01#'
    page = requests.get(full_link)
    soup = BeautifulSoup(page.text,'lxml')
    
    
    #Grab the state names
    states = soup.find_all('span',class_='fred-rls-elm-nm')
    states_list = []
    for j in states:
        state_name = j.text
        states_list.append(state_name)

    #Package into df
    states_df = pd.DataFrame(states_list,columns=['State'])
    states_df['index'] = np.arange(len(states_df))
    
    
    #Grab the population values
    bachs = soup.find_all('td',class_='fred-rls-elm-vl-td')
    bach_list = []
    for k in bachs:
        bach_num = k.text
        bach_list.append(bach_num)


    #Package into a df
    bach_df = pd.DataFrame(bach_list,columns=['Bach_Perc'])
    
    #Clean up population values
    bach_df['Bach_Perc'] = bach_df['Bach_Perc'].replace('\n',"",regex=True).str.strip()
    bach_df['Bach_Perc'] = bach_df['Bach_Perc'].replace(',',"",regex=True)
    #bach_df = bach_df.apply(pd.to_numeric,errors='ignore')
    #Grab every third value
    bach_df = bach_df.iloc[::3, :]
    #bach_df['Bach_Perc'] = (bach_df['Bach_Perc']*1000).astype(int)
    
    #Create index for joining
    bach_df['index'] = np.arange(len(bach_df))
    
    #Combine dataframes
    full_df = pd.merge(
        states_df,
        bach_df,
        how='left',
        on=['index']
    )
    year_pop = str(i)
    full_df['Year'] = year_pop
    
    
    list_of_dfs.append(full_df)
    
full_df = pd.concat(list_of_dfs)
full_df

,State,index,Bach_Perc,Year
0,Alabama,0,21.1,2006
1,Alaska,1,26.9,2006
2,Arizona,2,25.5,2006
3,Arkansas,3,18.2,2006
4,California,4,29,2006
...,...,...,...,...
46,Virginia,46,39.3,2018
47,Washington,47,36.7,2018
48,West Virginia,48,21.3,2018
49,Wisconsin,49,30.0,2018


In [17]:
full_df.to_csv('states_and_bachs.csv', index=False)